## Laden der Text Daten 

In [20]:
import os
import pandas as pd
import numpy as np
import urllib.request

DATADIR = "data"

if not os.path.exists(DATADIR): 
    os.mkdir(DATADIR)

file_name = os.path.join(DATADIR, "17-Bundestag.csv")
if not os.path.exists(file_name):
    url_data = 'https://www.dropbox.com/s/i4lp1fidpbo3k1w/17-Bundestag.csv'
    urllib.request.urlretrieve(url_data, file_name)

df = pd.read_csv(file_name)

print("{} Reden wurden geladen".format(len(df)))

# Zeige einige Reden von Parteien
for party in np.unique(df['partei']):
    print("\n****************************\nBeispiele von Partei {}:\n****************************\n".format(party))
    for _, speech in df[df['partei']==party].sample(2).iterrows():
        print("\n*******************************\nSprecher {}:\n\n{}".format(speech['sprecher'], speech['text']))


57636 Reden wurden geladen

****************************
Beispiele von Partei cducsu:
****************************


*******************************
Sprecher Peter Beyer:

Ich freue mich, dass allen Unkenrufen zum Trotz - ich nenne das Unwort vom angeblichen Continental Drift - die Bemühungen für die Idee einer umfassenden transatlantischen Wirtschaftszone Unterstützung auf allerhöchster politischer Ebene erfahren. Die Weiterentwicklung der transatlantischen Wirtschaftsintegration verspricht den Wachstums- und Jobmotor in Europa und in den USA weiter zu befeuern.

Es gibt Probleme, insbesondere im Bereich der nichttarifären Hemmnisse und bei den unterschiedlichen -Qualitäts- und Sicherheitsstandards bei Gütern, bei Dienstleistungen, beim Handel, bei Innovationen und Investitionen. Dies verursacht jedes Jahr beidseits des Atlantiks, in Europa wie in den USA, Kosten in mehrfacher Milliardenhöhe, die völlig unnötig sind. Die Verhandlungen zwischen der EU und Kanada über die Errichtung ein

In [76]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

def train(texts, party):
    '''
    Eine Funktion, die gegeben Texte und Labels einen Klassifier trainiert
    '''

    # Eine Machine Learning Pipeline um Bag-of-Words Vectors zu erstellen aus Texten
    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), max_df=0.5)),
                        ('clf', SGDClassifier())])

    text_clf.fit(texts,party)

    return text_clf


## Trainieren eines linearen Klassifiers

In [77]:
# Erst nehmen wir einige Daten beiseite, um das Model darauf spaeter testen zu koennen
train_data, test_data, train_labels, test_labels = train_test_split(df['text'], df['partei'], test_size=0.2)

# Jetzt trainieren wir den Klassifier
clf = train(df['text'], df['partei'])

/Users/felix/Code/Python/erlebe-it-ml/venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


## Performance auf Test Daten

In [78]:
test_predictions = clf.predict(test_data)
report = classification_report(test_predictions, test_labels)
print(report)

             precision    recall  f1-score   support

     cducsu       0.85      0.75      0.80      3718
        fdp       0.61      0.80      0.70      1311
     gruene       0.57      0.84      0.68      1278
      linke       0.83      0.70      0.76      2098
        spd       0.78      0.72      0.75      3123

avg / total       0.77      0.75      0.75     11528



## Confusion Matrix

In [79]:
print(confusion_matrix(test_labels, test_predictions))

[[2796  121   39  125  217]
 [ 376 1051   43   96  147]
 [ 186   48 1069  206  353]
 [  76   23   44 1473  155]
 [ 284   68   83  198 2251]]


## Download eines Zeitungsartikels

In [80]:
import urllib.request
from readability.readability import Document

url = "http://www.spiegel.de/politik/ausland/muss-deutschland-jetzt-milliarden-an-polen-zahlen-a-1167144.html"
html = urllib.request.urlopen(url).read()
readable_article = Document(html).summary()
readable_title = Document(html).short_title()


## Klassifizierung des Zeitungsartikels

In [81]:
prediction = clf.predict([readable_article])

print("Zeitungsartikel: {}".format(readable_title))
print("Partei Klassifikation: {}".format(prediction))

Zeitungsartikel: Muss Deutschland jetzt Milliarden an Polen zahlen?
Partei Klassifikation: ['linke']


In [83]:
bow_vectors = clf.steps[0][1].transform(df.text)
idx2words = {k:v for v,k in clf.steps[0][1].vocabulary_.items()}
for party in np.unique(df['partei']):
    this_party = (df['partei'] == party).values * 2 - 1
    word_covariance = bow_vectors.T.dot(this_party).argsort()
    top_words = [idx2words[widx] for widx in word_covariance[-20:][::-1]]
    print("*********************\nPartei {} benutzt haeufig:\n{}".format(party, ", ".join(top_words)))

*********************
Partei cducsu benutzt haeufig:
christlich, verehrten, kernenergie, dankbar, christen, wettbewerbsfähigkeit, währung, mitbürger, unionsgeführte, wiedervereinigung
*********************
Partei fdp benutzt haeufig:
liberaler, zuwanderer, zuwanderungssteuerung, zuwanderungspolitik, pflanzenzüchtung, zeitarbeiter, bereicherten, willkommens, beschäftigungsformen, geschätzter
*********************
Partei gruene benutzt haeufig:
garantierente, leiharbeitskräfte, baustoffe, bäuerliche, einwanderinnen, agrarindustrie, immigranten, einwanderer, menschenrechtskriterien, bündnisgrüne
*********************
Partei linke benutzt haeufig:
linke, erwerbslosen, profite, konzerne, südens, krieg, erwerbslose, waffenexporte, vermögensmillionäre, neoliberalen
*********************
Partei spd benutzt haeufig:
sozialdemokraten, sozialdemokratinnen, bundestagsfraktion, olaf, dat, michelbach, seehofer, entwicklungsminister, ganztagsschulen, steuersenkungen
